In [ ]:
from crewai import Crew, Task, Agent,Task, Process,LLM
from crewai_tools import SerperDevTool
import os
from dotenv import load_dotenv

load_dotenv()

web_search_tool = SerperDevTool(n_results=3,api_key=os.getenv("SERPER_API_KEY"))

llm = LLM(model="ollama/llama3.2:1b", temperature=0.7,
          base_url="http://localhost:11434")

researcher = Agent(
    role="Content Researcher",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory=("You're working on planning a blog article about the topic: {topic}. "
               "You collect information from the web and plan an outline " 
               "for an article that helps the audience learn something "
               "and make informed decisions. Your work is the basis for "
               "the Content Writer to write an article on this topic. "
               "Use the search tool to search for the most relevant and " 
               "factually accurate data about {topic}."),
    tools=[web_search_tool]
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate article about the topic: {topic}",
    backstory=("You're working on writing a new blog post about the topic: {topic}. "
               "You base your writing on the work of the Content Researcher, "
               "who provides an outline and relevant context about the topic. "
               "You follow the main objectives and direction of the outline, "
               "as provided by the Content Researcher. You provide objective and "
               "impartial insights and back them up with information "
               "provided by the Content Researcher. "),
)

editor = Agent(
    role="Content Editor",
    goal=("Edit a given blog post to align with a structured "
          "format and remove false and misleading information. "
          "Ensure all the information provided is factually correct. "
          "Use the search tool for finding up-to-date and correct information."),
    backstory=("You are an editor who receives a blog post from the Content Writer. "
               "Your goal is to review the blog post to ensure that it follows "
               "best practices and adheres to a structured format. "
               "Verify the accuracy of information from the web using the "
               "search tool ensuring only accurate information is published."),
    tools=[web_search_tool]
)


plan_task = Task(
    description="Outline the main sections and points for {topic}.",
    expected_output="""A structured outline with
                       sections and bullet points.""",
    agent=researcher,
    tools=[web_search_tool]
)

write_task = Task(
    description="Expand the outline into a detailed article.",
    expected_output="""A comprehensive article draft
                       based on the outline.""",
    agent=writer
)

edit_task = Task(
    description="""Edit the article draft for
                   clarity and correctness.""",
    expected_output="""A polished, final article
                       ready to be published.""",
    agent=editor,
)

crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[plan_task, write_task, edit_task],
    process=Process.sequential,
    verbose=True
)

TOPIC = "the future of space exploration"

result = crew.kickoff(inputs={"topic": TOPIC})


In [ ]:
print(result)